## Napovedovanje vrednosti

Podatkovno rudarjenje, naloga, `10. 5. 2019`  
**`Mitja Hrovatič`**

## Predicting values

Data mining, assignment, `<INSERT DATE>`  
**`<INSERT Name and surname>`**

Spoznali bomo praktično uporabo enostavnih metod nadzorovanega modeliranja oz.
napovedovanja.  Skupna lastnost vseh omenjenih metod je, da s pomočjo
naključnih spremenljivk (atributov) modelirajo vrednosti posebne spremenljivke,
ki ji pravimo *razred* (v kontekstu uvrščanja v razrede, klasifikacije)
ali *odziv* (v kontekstu regresije). Osnovne razlike med kontekstoma smo
spoznali na predavanjih in vajah.

Praktična cilja, ki ju bomo zasledovali sta:
* modeliranje ocen posameznega uporabnika (odziva) s pomočjo vseh ostalih uporabnikov,
* primerjava metod nadzorovanega modeliranja.

We will get to know the practical use of simple methods of supervised modeling or forecasting. The common property of all of these methods is that with the help of random variables (attributes) they model the values of a specific variable,
which we call *class* (in the context of classifying, classifying)
or *response* (in the context of regression). We learnt about the basic differences between contexts in lectures and tutorials.

The practical goals we will pursue are:
* modeling of individual user's responses (responses) with the help of all other users,
* Comparison of supervised modeling methods.

### Podatki

Opis podatkovne zbirke MovieLens 1996-2016 ostaja enak prvi nalogi.

In [1]:
from csv import DictReader
from datetime import datetime
import numpy as np
from scipy.stats import multivariate_normal as mvn 
import math
from operator import itemgetter, attrgetter
import itertools
from scipy.stats import beta
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_formats = ['jpg']
import matplotlib
matplotlib.figure.Figure.__repr__ = lambda self: (
    f"<{self.__class__.__name__} size {self.bbox.size[0]:g}"
    f"x{self.bbox.size[1]:g} with {len(self.axes)} Axes>")

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
plt.style.use('PR.mplstyle')


# Shranjeni podatki
movies = {}
zanr = {}
cast = {}
ratings_movie = {}
ratings_user = {}
tags = []
links = []
actors = {}
users = {}



with open('podatki/ml-latest-small/movies.csv', 'rt', encoding='utf-8') as data:
    for row in DictReader(data):
        movies[row['movieId']] = row
        for genre in row['genres'].split('|'):
            if genre not in zanr:
                zanr[genre] = [genre]
            else:
                zanr[genre] += [genre]
        
with open('podatki/ml-latest-small/ratings.csv', 'rt', encoding="utf-8") as data:
    for row in DictReader(data):
        if row['movieId'] not in ratings_movie:
            ratings_movie[row['movieId']] = []
        ratings_movie[row['movieId']].append([float(row['rating']), int(row['timestamp'])])
        
        if row['movieId'] not in ratings_user:
            ratings_user[row['movieId']] = []
        ratings_user[row['movieId']].append([int(row['userId']), float(row['rating'])])
        
        if row['userId'] not in users:
            users[row['userId']] = []
        users[row['userId']].append([float(row['rating']), row['movieId']])
        
    for key, value in ratings_movie.items():
        ratings_movie[key] = np.array(value)
    
    for key, value in ratings_user.items():
        ratings_user[key] = np.array(value)
    
    for key, value in users.items():
        users[key] = np.array(value)
        
    
        
with open('podatki/ml-latest-small/cast.csv', 'rt', encoding="utf-8") as data:
    for row in DictReader(data):
        cast[row['movieId']] = row['cast'].split("|")
        
    for key, value in cast.items():
        for actor in value:
            if actor not in actors:
                actors[actor] = []
            actors[actor] += [key]
        
with open('podatki/ml-latest-small/tags.csv', 'rt', encoding="utf-8") as data:
    for row in DictReader(data):
        tags.append(row)
        
with open('podatki/ml-latest-small/links.csv', 'rt', encoding="utf-8") as data:
    for row in DictReader(data):
        links.append(row)

### Data

The description of the MovieLens database remains the same as for the first homework.

### Predpriprava podatkov

Za potrebe te naloge bomo podatke pripravili na naslednji način:
1. Izberi $m$ filmov z vsaj 100 ogledi.
2. Izberi $n$ uporabnikov, ki si je ogledalo vsaj 100 filmov.
3. Pripravi matriko $X$ velikosti $m \times n$, kjer vrstice predstavljajo filme, stolpci pa uporabnike. Neznane vrednosti zamenjaj z $0$.

Za vsakega od izbranih $n$ uporabnikov bo zgrajen regresijski model, 
katerega cilj bo napoved ocen za filme. 

### Preparation of data

For the purposes of this task we will prepare the data as follows:
1. Select $m$ movies with at least 100 views.
2. Select $n$ users who have watched at least 100 movies.
3. Prepare matrix $X$ in the size of $m \times n$, where the lines represent movies and columns represent users. Replace unknown values with $0$.

For each of the selected $n$ users, a regression model will be built, which aims to predict film ratings.

<table>
    <tr style="background-color: white;">
        <td style="border-right: 1px solid #000;"></td>
        <td></td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">$y^{(0)}$</td>
        <td colspan=3 style="text-align:center;">$X^{(0)}$</td>
    </tr>
    <tr style="border-bottom: 1px solid #000;">
        <td style="border-right: 1px solid #000;"></td>
        <td>Film/uporabnik</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">$u_0$</td>
        <td>$u_1$</td>
        <td>$u_2$</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_1}$</td>
        <td>Twelve Monkeys (a.k.a. 12 Monkeys) (1995)</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">0</td>
        <td>0</td>
        <td>2.5</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_2}$</td>
        <td>Dances with  Wolves (1990) </td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">4</td>
        <td>0</td>
        <td>0</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_3}$</td>
        <td>Apollo 13 (1995)</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">0</td>
        <td>2</td>
        <td>0</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_4}$</td>
        <td>Sixth Sense, The (1999)</td><td style="border-right: 1px solid #000; border-left: 1px solid #000;">3</td>
        <td>0</td>
        <td>4</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">$\cdots$</td>
        <td>$\cdots$</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">$\cdots$</td>
        <td>$\cdots$</td>
        <td>$\cdots$</td>
        <td>$\cdots$</td>
    </tr>
</table>

<table>
    <tr style="background-color: white;">
        <td style="border-right: 1px solid #000;"></td>
        <td></td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">$y^{(1)}$</td>
        <td colspan=3 style="text-align:center;">$X^{(1)}$</td>
    </tr>
    <tr style="border-bottom: 1px solid #000;">
        <td style="border-right: 1px solid #000;"></td>
        <td>Film/uporabnik</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">$u_1$</td>
        <td>$u_0$</td>
        <td>$u_2$</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_1}$</td>
        <td>Twelve Monkeys (a.k.a. 12 Monkeys) (1995)</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">0</td>
        <td>0</td>
        <td>2.5</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_2}$</td>
        <td>Dances with  Wolves (1990) </td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">0</td>
        <td>4</td>
        <td>0</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_3}$</td>
        <td>Apollo 13 (1995)</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">2</td>
        <td>0</td>
        <td>0</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">${f_4}$</td>
        <td>Sixth Sense, The (1999)</td><td style="border-right: 1px solid #000; border-left: 1px solid #000;">0</td>
        <td>3</td>
        <td>4</td>
        <td>$\cdots$</td>
    </tr>
    <tr>
        <td style="border-right: 1px solid #000;">$\cdots$</td>
        <td>$\cdots$</td>
        <td style="border-right: 1px solid #000; border-left: 1px solid #000;">$\cdots$</td>
        <td>$\cdots$</td>
        <td>$\cdots$</td>
        <td>$\cdots$</td>
    </tr>
</table>

Razdelitev podatkov za model uporabnika $u_0$ (zgoraj) in uporabnika $u_1$ (spodaj).

Data distribution for the model user $u_0$ (above) and the user $u_1$ (below).

### Vprašanja

<ol>
<li>(100%) **Regresija**. Za vsakega od uporabnikov postavite regresijski model. Uporabite eno ali več metod za učenje regresijskih modelov (linearna regresija, Ridge, Lasso, itd.). <br>
    Za vsakega od $n$ uporabnikov izberite ustrezni stolpec v matriki podatkov. Za uporabnika $i$ imamo torej
    <ul>
    <li> Vektor odziva $y^{(i)}$,</li>
    <li> Matriko podatkov $X^{(i)}$, ki vsebuje vse stolpce *razen* $i$.</li>
    </ul>
    
Za lažjo predstavo si oglej zgornji tabeli. Nekajkrat (npr. trikrat) ponovite postopek preverjanja s pomočjo učne in testne množice:

<ol type="a">
    <li> Množico filmov, ki si jih je uporabnik ogledal, *naključno* razdelite v razmerju 75% (učna množica) in 25% (testna množica).</li>
    <li> Naučite regresijski model na učni množici<br> (izberite ustrezne vrstice v $X$ in $y$).</li>
    <li> Ovrednotite model na testni množici <br> (ponovno izberite ustrezne vrstice v $X$ in $y$).</li>
</ol>
Oceno vrednotenja nato delite s številom poizkusov, da dobite končno oceno.<br>

Poročajte o uspešnosti vašega modela. Pri tem se osredotočite na naslednja vprašanja:
<ul>
    <li> Utemeljite ustrezno mero vrednotenja. Ali model dobro napoveduje ocene?
    <li> Z izbrano mero ocenite modele za vseh $n$ uporabnikov.
</ul>

</li>


<li>
(Bonus 15%) Ustvarite novega uporabnika, ki predstavlja vaše ocene
filmov. Ocenite nekaj filmov po lastnem okusu in preverite, kako modeli ocenijo neizbrane filme.
Ali se vam zdijo napovedi primerne? 
</li>
</ol>

### Questions

<ol>
<li> (100%) **Regression**. Set up a regression model for each user. Use one or more methods for learning regression models (linear regression, Ridge, Lasso, etc.). <br>
    For each of the $n$ users, select the appropriate column in the data matrix. Therefore, fot the $i$ user we have
    <ul>
    <li> Response vector $y^{(i)}$, </li>
    <li> Data matrix $X^{(i)}$ containing all columns *except* $i$.</li>
    </ul>
    
For an easier understanding see the above tables. Repeat the test procedure several times (eg three times) with the help of the learning and test sets:

    <ol type="a">
    <li> Divide the films that the user viewed *randomly* into 75% (learning set) and 25% (test set).</li>
    <li> Learn the regression model on learning set<br>(select the appropriate rows in $X$ and $y$).</li>
    <li> Evaluate the model on the test set <br> (select the appropriate rows in $X$ and $y$).</li>
    </ol>
Then divide the evaluation score with the number of experiments to get the final score.<br>

Report on the performance of your model. Focus on the following questions:
<ul>
    <li> Justify an appropriate evaluation score. Does the predict the scores well?
    <li> Rate the models for all $n$ users with the selected evaluation score.
</ul>
</li>

<li>
(Bonus 15%) Create a new user who represents your movie ratings. Rate some movies after your own taste and see how models evaluate non-selected movies. Do you find the predictions appropriate?
</li>
</ol>

### Zapiski

Implementacijo, opis in vrednotenje metod za nadzorovanjo učenje vsebujejo knjižnice `sklearn` ali `Orange`.

In [2]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

In [3]:
matrika_film_ocene = {}
list_of_matrika_film_ocene = []
# list_of_title_film = []

for title, value in ratings_movie.items():
    ocene_filma = value[:, 0]
    n = len(ocene_filma)
    if n >= 100:
        if title not in matrika_film_ocene:
            matrika_film_ocene[title] = []
        for key, value in users.items():
            if len(value) >= 100:
                for user_rate, movie_of in value:
                    if movie_of == title:
                        matrika_film_ocene[title] += [float(user_rate)]
                        break
                else:
                    matrika_film_ocene[title] += [np.mean(ocene_filma)]


print("Število filmov:", len(matrika_film_ocene))
print("Število uporabnikov:", len(matrika_film_ocene['10']))


for key, value in matrika_film_ocene.items():
    list_of_matrika_film_ocene.append(value)
#     list_of_title_film.append(movies[key]['title'])
#     list_of_title_film.append(float(key))


list_of_matrika_film_ocene = np.array(list_of_matrika_film_ocene)

print("Dimenzije matrike:", list_of_matrika_film_ocene.shape)
print("Št. elementov:", list_of_matrika_film_ocene.size)

Število filmov: 151
Število uporabnikov: 263
Dimenzije matrike: (151, 263)
Št. elementov: 39713


In [4]:
vektor_odziva_uporabniki = []
matrike_podatkov = []

for i in range(len(list_of_matrika_film_ocene[0])):
    vektor_odziva_uporabniki += [list_of_matrika_film_ocene[:, [i]]]
    matrike_podatkov += [np.append(list_of_matrika_film_ocene[:, :i], list_of_matrika_film_ocene[:, i+1:], axis=1)]
    
# print(len(matrike_podatkov))
# print(len(vektor_odziva_uporabniki))

### Učenje regresijskih modelov

In [5]:
def ucenje(model_of):
    mse = []
    mae = []
#     meae = []

    # Ucenje modela
    for i in range(len(list_of_matrika_film_ocene[0])):
        _mse = []
        _mae = []
        for j in range(3):
            X_train, X_test, y_train, y_test = train_test_split(matrike_podatkov[i], vektor_odziva_uporabniki[i], test_size=0.25) # , random_state=42

            x = X_train
            y = y_train

            model = model_of
            model.fit(x, y)

            # Napoved vrednosti za podatke
            hx = model.predict(X_test)

            _mse.append(mean_squared_error(y_test, hx))
            _mae.append(mean_absolute_error(y_test, hx))
    #         meae.append(median_absolute_error(y_test, hx))
        mse.append(np.mean(_mse))
        mae.append(np.mean(_mae))
        

    # print(model.intercept_,model.coef_)

    print("MSE", np.mean(mse))
    print("MAE", np.mean(mae))
#     print("MEDIAN AE", np.mean(meae))

#### Linearna regresija

In [6]:
# Linearna regresija

ucenje(LinearRegression())

MSE 0.49741669859396787
MAE 0.5033116619702638


#### Lasso regresija

In [7]:
# Lasso regresija, izračun z merami za napake na vseh modelih, nato pa izpišem povprečje vseh modelov

ucenje(Lasso(alpha=0.1))

MSE 0.32528645493587466
MAE 0.3765138919030429


#### Ridge regresija

In [8]:
# Ridge regresija

ucenje(Ridge(alpha=0.1))

MSE 0.4942096157399291
MAE 0.5035100473188479


### Notes

Implementation, description, and evaluation of supervised learning methods are included in libraries `skleaarn` or` Orange`.

### Odgovor
**Uporabil sem tri metode za učenje regresijski modelov (linearna regresija, Ridge, Lasso). Ugotovil sem, da je najboljši model za napovedovanje, model Lasso, saj sem prišel do ugodovitve z merami za napake (MSE, MAE), da ima najmanjše napake.
Model dobro napoveduje ocene, saj ima odstopanja pri MSE samo ~0.32 ocene.**